# NOTEBOOK 1 : Découverte et Création des Catégories

**Objectif :** Ce notebook analyse un **gros fichier CSV** de dépôts GitHub pour en extraire des catégories de manière non supervisée, **sans saturer la mémoire**.

1.  Il lit le CSV par **petits morceaux (chunks)**.
2.  Il génère des embeddings pour chaque morceau.
3.  Il utilise **`MiniBatchKMeans`** pour apprendre les catégories de manière incrémentale.
4.  Il relit le fichier pour assigner les catégories et collecter les textes pertinents.
5.  Il nomme et sauvegarde les catégories dans un fichier JSON.

In [1]:
!pip install pandas numpy torch transformers sentence-transformers scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 8.4 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 2.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.2 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.0 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.1/802.1 kB 4.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [sentence-transformers]ence-transformers]


In [2]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
import gc

# Machine Learning & Embeddings
import torch
from sentence_transformers import SentenceTransformer
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

# --- CONFIGURATION ---

# Fichiers
INPUT_CSV_FILE = "github_data_with_readmes.csv"
OUTPUT_CATEGORIES_FILE = "github_categories_database.json"

# Paramètres de traitement
CHUNK_SIZE = 2000  # Nombre de lignes à traiter en mémoire à la fois. Ajustez selon votre RAM.
N_CATEGORIES = 200
EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Utilisation du device : {device}")

2025-10-13 07:44:11.638402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Utilisation du device : cpu


### Phase 1 : Entraînement du Modèle de Clustering

Ici, nous faisons une première passe sur le fichier. L'objectif est uniquement d'**entraîner le modèle `MiniBatchKMeans`** à reconnaître les `N_CATEGORIES` groupes de dépôts. Nous ne stockons rien en mémoire à part le modèle lui-même.

In [ ]:
print("--- Phase 1: Entraînement du modèle de clustering ---")

# 1. Initialiser le modèle de clustering incrémental
kmeans = MiniBatchKMeans(
    n_clusters=N_CATEGORIES,
    random_state=42,
    batch_size=256, # Taille des lots pour l'entraînement interne de KMeans
    n_init='auto'
)

# 2. Initialiser le modèle d'embedding
embedding_model = SentenceTransformer(EMBEDDING_MODEL, device=device)

# 3. Créer un lecteur de CSV par morceaux
csv_reader = pd.read_csv(INPUT_CSV_FILE, chunksize=CHUNK_SIZE, iterator=True)

# 4. Boucle d'entraînement sur chaque morceau du fichier
for chunk in tqdm(csv_reader, desc="Entraînement du clustering"):
    # Préparation du texte pour le lot actuel
    chunk['description'] = chunk['description'].fillna('')
    chunk['readme_content'] = chunk['readme_content'].fillna('')
    chunk['full_text'] = chunk['description'] + ' ' + chunk['readme_content']
    
    # Filtrer les textes trop courts
    chunk = chunk[chunk['full_text'].str.strip().str.len() > 50]
    if chunk.empty:
        continue

    # Génération des embeddings pour le lot
    embeddings = embedding_model.encode(
        chunk['full_text'].tolist(), 
        show_progress_bar=False, # On a déjà la barre de progression principale
        batch_size=128
    )
    
    # Entraînement partiel du modèle sur ce lot
    kmeans.partial_fit(embeddings)
    
    # Libérer la mémoire
    del chunk, embeddings
    gc.collect()

print("✅ Modèle de clustering entraîné. Les prototypes des catégories sont prêts.")
category_prototypes = kmeans.cluster_centers_

--- Phase 1: Entraînement du modèle de clustering ---


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Entraînement du clustering: 0it [00:00, ?it/s]

### Phase 2 : Assignation des Catégories et Nommage

Maintenant que le modèle est entraîné, nous faisons une **deuxième passe** sur le fichier. Cette fois, l'objectif est de :
1. Prédire à quelle catégorie appartient chaque dépôt.
2. Collecter les textes de chaque catégorie pour pouvoir les nommer.

In [ ]:
print("\n--- Phase 2: Assignation et nommage des catégories ---")

# 1. Préparer des listes pour stocker les textes de chaque catégorie
texts_per_category = [[] for _ in range(N_CATEGORIES)]
repos_per_category = [0] * N_CATEGORIES

# 2. Recréer le lecteur de CSV pour la deuxième passe
csv_reader_pass2 = pd.read_csv(INPUT_CSV_FILE, chunksize=CHUNK_SIZE, iterator=True)

for chunk in tqdm(csv_reader_pass2, desc="Assignation & Collecte de texte"):
    # Préparation du texte (identique à la phase 1)
    chunk['description'] = chunk['description'].fillna('')
    chunk['readme_content'] = chunk['readme_content'].fillna('')
    chunk['full_text'] = chunk['description'] + ' ' + chunk['readme_content']
    chunk = chunk[chunk['full_text'].str.strip().str.len() > 50]
    if chunk.empty:
        continue

    # Génération des embeddings
    embeddings = embedding_model.encode(chunk['full_text'].tolist(), show_progress_bar=False, batch_size=128)
    
    # Prédiction des catégories pour ce lot
    labels = kmeans.predict(embeddings)
    
    # Ajout des textes dans les listes correspondantes
    for text, label in zip(chunk['full_text'], labels):
        # On ne garde qu'un échantillon de textes par catégorie pour ne pas saturer la RAM
        if repos_per_category[label] < 200: # Échantillon de 200 dépôts max par catégorie pour le nommage
            texts_per_category[label].append(text)
        repos_per_category[label] += 1
        
    del chunk, embeddings, labels
    gc.collect()

print("✅ Textes collectés pour chaque catégorie.")

# 3. Nommage des catégories en utilisant TF-IDF sur les textes collectés
print("Nommage des catégories...")
categories_database = []
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, max_df=0.8, min_df=2)

for i in tqdm(range(N_CATEGORIES), desc="Finalisation des catégories"):
    # S'assurer qu'il y a assez de texte pour vectoriser
    if len(texts_per_category[i]) < 5:
        category_name = f"Cluster {i} - small_or_generic"
    else:
        tfidf_matrix = vectorizer.fit_transform(texts_per_category[i])
        terms = vectorizer.get_feature_names_out()
        mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).ravel()
        top_indices = mean_tfidf.argsort()[-5:][::-1]
        keywords = [terms[j] for j in top_indices]
        category_name = f"Cluster {i} - {', '.join(keywords)}"

    categories_database.append({
        "category_id": i,
        "category_name": category_name,
        "embedding_prototype": category_prototypes[i].tolist()
    })

# 4. Sauvegarde finale
with open(OUTPUT_CATEGORIES_FILE, 'w', encoding='utf-8') as f:
    json.dump(categories_database, f, ensure_ascii=False, indent=4)

print(f"\n✅ Base de {N_CATEGORIES} catégories créée et sauvegardée dans '{OUTPUT_CATEGORIES_FILE}'")